In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging

from madminer.core import MadMiner
from madminer.plotting import plot_2d_morphing_basis
from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.sampling import SampleAugmenter
from madminer.sampling import constant_benchmark_theta, multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge


In [2]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## 7. Train neural networks to estimate likelihood ratios

It's now time to build the neural network that estimates the likelihood ratio. The central object for this is the `madminer.ml.MLForge` class. It defines functions that train, save, load, and evaluate the estimators.

In [3]:
forge = MLForge()

We will use the RASCAL method described in ["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013) and ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020). Other implemented methods include CARL, CASCAL, and ROLR described in the same publications. There is also SCANDAL introduced in ["Mining gold from implicit models to improve likelihood-free inference"](https://arxiv.org/abs/1805.12244) as well as ALICE and ALICES which are introduced in ["Likelihood-free inference with an improved cross-entropy estimator"](https://arxiv.org/abs/1808.00973).

Most of these methods exist both in a "single parameterized" version, in which only the dependence of the likelihood ratio on the numerator is modelled, and a "doubly parameterized" version, in which both the dependence on the numerator and denominator parameters is modelled. For the single parameterized version, use `method='rascal'`, `method='alice'`, and so on. For the double parameterized version, use `method='rascal2'`, `method='alice2'`, etc. Note that for the doubly parameterized estimators you have to provide `theta1_filename`, and in the case of RASCAL and ALICE also `t_xz1_filename`.

In [6]:
forge.train(
    method='alice',
    theta0_filename='data/samples/theta0_train1.npy',
    x_filename='data/samples/x_train1.npy',
    y_filename='data/samples/y_train1.npy',
    r_xz_filename='data/samples/r_xz_train1.npy',
    t_xz0_filename='data/samples/t_xz_train1.npy',
    n_hidden=(20,20,20),
#    alpha=5.,
#    initial_lr=0.1,
    n_epochs=20,
    validation_split=0.3,
    batch_size=512
)

forge.save('models/model')

16:22 madminer.ml          INFO    Starting training
16:22 madminer.ml          INFO      Method:                 alice
16:22 madminer.ml          INFO      Training data:          x at data/samples/x_train1.npy
16:22 madminer.ml          INFO                              theta0 at data/samples/theta0_train1.npy
16:22 madminer.ml          INFO                              y at data/samples/y_train1.npy
16:22 madminer.ml          INFO                              r_xz at data/samples/r_xz_train1.npy
16:22 madminer.ml          INFO                              t_xz (theta0) at data/samples/t_xz_train1.npy
16:22 madminer.ml          INFO      Features:               all
16:22 madminer.ml          INFO      Method:                 alice
16:22 madminer.ml          INFO      Hidden layers:          (20, 20, 20)
16:22 madminer.ml          INFO      Activation function:    tanh
16:22 madminer.ml          INFO      Batch size:             512
16:22 madminer.ml          INFO      Trainer:       

We continue with the validation of the trained model.